<div class="alert alert-block alert-success"><font color="black"><h1><left>Setup Work For MIMIC Model Creation For Intestinal Model</left></h1></font><P>
<font size="5">- Balance Outcomes</font><P>
<font size="5">- Train/Test/Val Split</font></div>

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
# Notebook Global Variables
icd10_class_def = [['I27.89'], ['K21.9']]

In [3]:
path = 'E:\Education\DataScience\CCSU-Data-Science\Data-Mining-And-Predictive-Analytics\Data-532\Data\\'
file1 = path + 'pd_notes_clean.snappy.parquet'
file2 = path + 'discharge_notes_clean.snappy.parquet'
file3 = path + 'base_adms.snappy.parquet'
file4 = path + 'pd_notes.snappy.parquet'

In [4]:
# Read data and examine counts
# Consistency between pn_notes and bas_adms shows I can eliminate base adm processing now that 
# segmentizing was removed from here and delegated to Hugging Face API
pn_notes = pd.read_parquet(file1,engine='auto',use_nullable_dtypes=True)
print('Admissions in pn notes: ', pn_notes.shape[0], 'Unique admissions: ', len(pn_notes['hadm_id'].unique()))
# print(pn_notes[0:3])
discharge_notes = pd.read_parquet(file2,engine='auto',use_nullable_dtypes=True)
print('Admissions in discharge notes: ', discharge_notes.shape[0], 'Unique discharge admissions: ', len(discharge_notes['hadm_id'].unique()))
# discharge_notes[0:3]
base_adms = pd.read_parquet(file3,engine='auto',use_nullable_dtypes=True)
print('Admissions in base notes: ', base_adms.shape[0], 'Unique base admissions: ', len(base_adms['hadm_id'].unique()))
# base_adms[0:3]



Admissions in pn notes:  2694 Unique admissions:  2694
Admissions in discharge notes:  6889 Unique discharge admissions:  6274
Admissions in base notes:  2694 Unique base admissions:  2694


In [5]:
pn_notes.columns

Index(['subject_id', 'hadm_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length'],
      dtype='object')

In [6]:
# count output diagnosis codes
pn_notes.groupby('primary_icd10_code')['hadm_id'].count()

primary_icd10_code
I27.89     669
K21.9     1209
K25.0       23
K25.4       65
K29.90      62
K29.91      41
K51.00       6
K51.90      58
K52.9       54
K56.60      71
K56.69      42
K58.9       33
K92.2      361
Name: hadm_id, dtype: int64

In [7]:
class_a = pn_notes[pn_notes.primary_icd10_code.isin(icd10_class_def[0])].hadm_id
class_b = pn_notes[pn_notes.primary_icd10_code.isin(icd10_class_def[1])].hadm_id
class_c = pn_notes[pn_notes.primary_icd10_code =='K92.2'].hadm_id
class_d = pn_notes[pn_notes.primary_icd10_code =='K25.4'].hadm_id
print(class_a.shape)
print(class_b.shape)
print(class_c.shape)
print(class_d.shape)


(669,)
(1209,)
(361,)
(65,)


In [8]:
#icds = [item for sublist in icd10_class_def for item in sublist]
pn_notesb = pn_notes[pn_notes.note_length < 30000]
pn_notesb.groupby(['primary_icd10_code'])\
                 .agg(min=('note_length', 'min'), median=('note_length','median'),\
                  max=('note_length','max') , count=('hadm_id','count'))

,min,median,max,count
primary_icd10_code,,,,
I27.89,397,8057.0,29518,588
K21.9,235,5740.0,29914,1120
K25.0,1572,5651.0,17853,23
K25.4,1002,4506.0,27998,60
K29.90,495,7058.0,27195,60
K29.91,743,5400.0,16871,39
K51.00,1815,13809.0,22736,5
K51.90,792,8132.0,25230,53
K52.9,840,5494.0,29074,47


In [9]:
# Opposed to other authors I did splitting first
# I am using base_adms here becuase scrub test split notes into pieces (multiple records per admiss)

class_a_id = pn_notes[pn_notes.primary_icd10_code.isin(icd10_class_def[0])].hadm_id
class_b_id = pn_notes[pn_notes.primary_icd10_code.isin(icd10_class_def[1])].hadm_id

# split train/test/val
# test
class_a_test = class_a_id.sample(frac=0.1, random_state=1)
class_b_test = class_b_id.sample(frac=0.1, random_state=1)


# train
class_a_train = class_a_id.drop(class_a_test.index)
class_b_train = class_b_id.drop(class_b_test.index)

# split test and val
class_a_val = class_a_test.sample(frac=0.5, random_state=1)
class_a_test = class_a_test.drop(class_a_val.index)
class_b_val = class_b_test.sample(frac=0.5, random_state=1)
class_b_test = class_b_test.drop(class_b_val.index)


In [10]:
print(class_a_train.shape)
print(class_a_test.shape)
print(class_a_val.shape)
print(class_b_train.shape)
print(class_b_test.shape)
print(class_b_val.shape)


(602,)
(33,)
(34,)
(1088,)
(61,)
(60,)


In [11]:
# Rebalance

class_a_train_rs = class_a_train.sample(n=500, replace=False, random_state=1)
class_b_train_rs = class_b_train.sample(n=500, replace=False, random_state=1)

class_a_test_rs = class_a_test.sample(n=30, replace=False, random_state=1)
class_b_test_rs = class_b_test.sample(n=30, replace=False, random_state=1)

class_a_val_rs = class_a_val.sample(n=30, replace=False, random_state=1)
class_b_val_rs = class_b_val.sample(n=30, replace=False, random_state=1)

In [12]:
class_b_val_rs[1:5]

1807    144328
1904    164175
2610    157880
1132    191491
Name: hadm_id, dtype: Int64

In [13]:
# check balancing

df_list = [class_a_train_rs, class_b_train_rs]
print('train')
for x in df_list:
    print('size : ', len(x))


df_listb = [class_a_test_rs, class_b_test_rs]
print('test')
for x in df_listb:
    print('size : ', len(x))

df_listc = [class_a_val_rs, class_b_val_rs]
print('val')
for x in df_listc:
    print('size : ', len(x))


train
size :  500
size :  500
test
size :  30
size :  30
val
size :  30
size :  30


In [14]:
# concatenate and test size and look for any overlap between test and train
train_df_list = [class_a_train_rs, class_b_train_rs]
id_train_df = pd.concat(train_df_list)

test_df_list = [class_a_test_rs, class_b_test_rs]
id_test_df = pd.concat(test_df_list)

val_df_list = [class_a_val_rs, class_b_val_rs]
id_val_df = pd.concat(val_df_list)

print('train size: ', id_train_df.shape[0])
print('test size: ', id_test_df.shape[0])
print('val size: ', id_val_df.shape[0])
 
# check between train and test
(pd.Index(id_test_df).intersection(pd.Index(id_train_df))).values

# print('between 25.0')
# (pd.Index(k25_0_train).intersection(pd.Index(k25_0_test))).values

# print('between 25.4')
# (pd.Index(k25_4_train).intersection(pd.Index(k25_4_test))).values

# print('between 29.90')
# (pd.Index(k29_90_train).intersection(pd.Index(k29_90_test))).values

# print('between 29.91')
# (pd.Index(k29_91_train).intersection(pd.Index(k29_91_test))).values

# print('between 51.00')
# (pd.Index(k51_00_train).intersection(pd.Index(k51_00_test))).values

# print('between 51.90')
# (pd.Index(k51_90_train).intersection(pd.Index(k51_90_test))).values

# print('between 52.9')
# (pd.Index(k52_9_train).intersection(pd.Index(k52_9_test))).values

# print('between 56.60')
# (pd.Index(k56_60_train).intersection(pd.Index(k56_60_test))).values

# print('between 56.69')
# (pd.Index(k56_69_train).intersection(pd.Index(k56_69_test))).values

# print('between 58.9')
# (pd.Index(k58_9_train).intersection(pd.Index(k58_9_test))).values




train size:  1000
test size:  60
val size:  60


array([], dtype=object)

In [15]:
print(id_train_df.shape)
print(pn_notes.shape)
len(pn_notes['hadm_id'].unique())
pn_notes.columns

(1000,)
(2694, 19)


Index(['subject_id', 'hadm_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length'],
      dtype='object')

In [16]:
pn_notes = pn_notes.rename(columns={'hadm_id': 'id'})
pn_notes.columns

Index(['subject_id', 'id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length'],
      dtype='object')

In [17]:
pn_notes
def create_label(text):
    
    if text in icd10_class_def[0]:
        temp_val = 0
    elif text in icd10_class_def[1]:
        temp_val = 1
    else:
        temp_val = 99
    
    return(temp_val)


id_train_df = pd.DataFrame(id_train_df)
intsl_train = pd.merge(id_train_df, pn_notes, left_on=['hadm_id'], right_on=['id'], how='left').drop(columns = ['id'])
#intsl_train = intsl_train.rename(columns={'hadm_id_x': 'hadm_id'})
#intsl_train = intsl_train.drop('hadm_id_y', axis = 1)
intsl_train['label'] = intsl_train['primary_icd10_code'].map(create_label).astype('category')
intsl_train['hadm_id'] = intsl_train['hadm_id'].astype('Int64')
intsl_train['text'] = intsl_train['text'].astype('string')
print('train', intsl_train.columns)

id_test_df = pd.DataFrame(id_test_df)
intsl_test = pd.merge(id_test_df, pn_notes, left_on=['hadm_id'], right_on=['id'], how='left').drop(columns = ['id'])
# intsl_test = intsl_test.rename(columns={'hadm_id_x': 'hadm_id'})
# intsl_test = intsl_test.drop('hadm_id_y', axis = 1)
intsl_test['label'] = intsl_test['primary_icd10_code'].map(create_label).astype('category')
intsl_test['hadm_id'] = intsl_test['hadm_id'].astype('Int64')
intsl_test['text'] = intsl_test['text'].astype('string')

print('test', intsl_test.columns)

id_val_df = pd.DataFrame(id_val_df)
intsl_val = pd.merge(id_val_df, pn_notes, left_on=['hadm_id'], right_on=['id'], how='left').drop(columns = ['id'])
#intsl_val = intsl_val.rename(columns={'hadm_id_x': 'hadm_id'})
#intsl_val = intsl_val.drop('hadm_id_y', axis = 1)
intsl_val['label'] = intsl_val['primary_icd10_code'].map(create_label).astype('category')
intsl_val['hadm_id'] = intsl_val['hadm_id'].astype('Int64')
intsl_val['text'] = intsl_val['text'].astype('string')
intsl_val['label'] = intsl_val['label'].astype('category')
print('val', intsl_val.columns)

# early_test.to_csv('./data/3days/test.csv')

train Index(['hadm_id', 'subject_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length', 'label'],
      dtype='object')
test Index(['hadm_id', 'subject_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length', 'label'],
      dtype='object')
val Index(['hadm_id', 'subject_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length

In [18]:
intsl_train = intsl_train[['hadm_id', 'text', 'primary_icd10_code', 'label']]\
                     .rename(columns={'hadm_id':'id', 'primary_icd10_code':'icd10_code'})
intsl_test = intsl_test[['hadm_id', 'text', 'primary_icd10_code', 'label']]\
                     .rename(columns={'hadm_id':'id', 'primary_icd10_code':'icd10_code'})
intsl_val = intsl_val[['hadm_id', 'text', 'primary_icd10_code', 'label']]\
                     .rename(columns={'hadm_id':'id', 'primary_icd10_code':'icd10_code'})
print(intsl_train[1:5])
print(intsl_test[1:5])
print(intsl_val[1:5])

       id                                               text icd10_code label
1  105739  yo male with hiv, phtn, hep c cirrhosis, osa, ...     I27.89     0
2  115732  encephalitis assessment: pt remains unresponsi...     I27.89     0
3  161874  events: ct head, spine- ct f/u shd- no acute b...     I27.89     0
4  156512  ccu nsg admit/progress note a-a/ s/p ami s- " ...     I27.89     0
       id                                               text icd10_code label
1  182294  tachycardic w labile bp requiring volume for l...     I27.89     0
2  115665  resp care pt. is a yo woman who felt sob, call...     I27.89     0
3  198985  nursing progress note o: yo women admitted fro...     I27.89     0
4  124458  respiratory care: patient transferred from osh...     I27.89     0
       id                                               text icd10_code label
1  120917  prob: s/p cabg x, iabp cv: st with occ pvcs no...     I27.89     0
2  130947  yr old with worsening o sats and sob at nursin...    

In [19]:
print(intsl_train.groupby('label')['id'].count())
print(intsl_test.groupby('label')['id'].count())
print(intsl_val.groupby('label')['id'].count())

label
0    500
1    500
Name: id, dtype: int64
label
0    30
1    30
Name: id, dtype: int64
label
0    30
1    30
Name: id, dtype: int64


In [20]:
print(intsl_train.dtypes)
intsl_train[1:5]

id               Int64
text            string
icd10_code      string
label         category
dtype: object


,id,text,icd10_code,label
1,105739,"yo male with hiv, phtn, hep c cirrhosis, osa, ...",I27.89,0
2,115732,encephalitis assessment: pt remains unresponsi...,I27.89,0
3,161874,"events: ct head, spine- ct f/u shd- no acute b...",I27.89,0
4,156512,"ccu nsg admit/progress note a-a/ s/p ami s- "" ...",I27.89,0


In [21]:
"""  This is extra as an example of pansql

import pandasql as ps

#qintsl = intsl_notes[['subject_id', 'hadm_id']]
# print(qintsl.dtypes)
a1 = "SELECT avg(text_length) FROM \ 
        (select len(text) as text_len from id_train_df) as a "
        

l1 = ps.sqldf(a1, locals())
print(l1) """

'  This is extra as an example of pansql\n\nimport pandasql as ps\n\n#qintsl = intsl_notes[[\'subject_id\', \'hadm_id\']]\n# print(qintsl.dtypes)\na1 = "SELECT avg(text_length) FROM \\ \n        (select len(text) as text_len from id_train_df) as a "\n        \n\nl1 = ps.sqldf(a1, locals())\nprint(l1) '

In [22]:
intsl_train[['id','text', 'icd10_code','label']].groupby(['icd10_code', 'label'])['id'].count()


icd10_code  label
I27.89      0        500
            1          0
K21.9       0          0
            1        500
Name: id, dtype: int64

<div class="alert alert-block alert-success"><font color="black"><h1><left>Write File with Admission Details and Attached Combined Notes</left></h1></font></div>

In [23]:
path = 'E://Education//DataScience/CCSU-Data-Science/Data-Mining-And-Predictive-Analytics/Data-532/Data/'
intsl_train.to_parquet(path + 'intsl_train_reflex_heart.snappy.parquet',compression='snappy',index=None)  
intsl_test.to_parquet(path + 'intsl_test_reflex_heart.snappy.parquet',compression='snappy',index=None)  
intsl_val.to_parquet(path + 'intsl_val_reflex_heart.snappy.parquet',compression='snappy',index=None)  


In [24]:
import boto3, os
s3 = boto3.resource('s3')
path = 'E://Education//DataScience/CCSU-Data-Science/Data-Mining-And-Predictive-Analytics/Data-532/Data/'
fname = 'intsl_train_reflex_heart.snappy.parquet'
f2name = 'intsl_test_reflex_heart.snappy.parquet'
f3name = 'intsl_val_reflex_heart.snappy.parquet'
S3_path = 'MIMIC/Data/' + fname
S3_path2 = 'MIMIC/Data/' + f2name
S3_path3 = 'MIMIC/Data/' + f3name
out_file = path + fname
out_file2 = path + f2name
out_file3 = path + f3name

In [25]:
s3.meta.client.upload_file(out_file, 'depratti-masters-thesis', S3_path)
s3.meta.client.upload_file(out_file2, 'depratti-masters-thesis', S3_path2)
s3.meta.client.upload_file(out_file3, 'depratti-masters-thesis', S3_path3)

In [26]:
print(intsl_train.shape)
print(intsl_test.shape)
print(intsl_val.shape)

(1000, 4)
(60, 4)
(60, 4)
